In [1]:
import boto3
from botocore.exceptions import ClientError
import os 
from os.path import join,exists
from os import makedirs
os.chdir('../')


In [2]:
from utils.load_session import load_session

In [3]:
credential,session=load_session()

# Amazon S3 (Simple Storage Service) and Buckets 

It is a scalable object storage service provided by AWS. This service can be used to store and retrive data from the web. 

`Buckets` is a Amazon S3 containers for storing objects(files). Each bucket is unique within a region, and globally. Bucket can store an unlimited number of objects. 

**Rules fore creating bucket name**
- The bucket name can be between 3 and 63 characters long, and can contain only lower-case characters, numbers, periods, and dashes.

- Each label in the bucket name must start with a lowercase letter or number.

- The bucket name cannot contain underscores, end with a dash, have consecutive periods, or use dashes adjacent to periods.

- The bucket name cannot be formatted as an IP address (198.51.100.24).

In [4]:
### creating S3 client 
## client is an instant object of the API 
## here s3_client is an instant of S3 service API
s3_client=session.client('s3')

In [5]:
# We can also initiate s3 client as boto3.client() 
# s3_client=boto3.client('s3',
#                        aws_access_key_id=credential['Access key ID'].item(),
#                         aws_secret_access_key=credential['Secret access key'].item(),
#                         region_name=credential['region'].item())

In [6]:
## creating a new bucket

# Create a new bucket
bucket_name = 'akashghimireofficial'
try:
    response = s3_client.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={
            'LocationConstraint': credential['region'].item()  # Specify your region
        }
    )
    print("Bucket created successfully")
except ClientError as e:
    if e.response['Error']['Code'] == 'BucketAlreadyExists':
        print("Bucket name is already taken. Please choose a different name.")
    elif e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
        print("You already own this bucket.")
    else:
        print(f"Error: {e}")

You already own this bucket.


In [7]:
response = s3_client.list_buckets()
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  akashghimireofficial


## Uploading File using S3 AWS service 

In order to upload file using s3 services we use `s3_client.upload_file()`

**Parameters**
- ``Filename``(**string**): Can be absolute or relative file path to be uploaded

- ``Bucket``(**string**): Name of the Bucket where you want to upload the file. In above example we created a unique-bucket

- ``Key``(**string**): The name you want to assign to the uploaded file in the bucket. It can as simple filename like 'filename.txt' or path like 'example_path/data/filename.txt'. The same key is used when retriving the file. 

In [8]:
from glob import glob 

In [10]:
for img_filename in glob(join('data','images','*')):
    key_path=join('images',os.path.basename(img_filename))
    s3_client.upload_file(Filename=img_filename,
                          Bucket=bucket_name,
                          Key=key_path)

## Seeing what's in the Bucket (Verifying Uploaded Files)

In order to see what is within the Bucket we can use the method:
 
prefix = 'folder/subfolder/' <br>

response = s3_client.``list_objects_v2(Bucket=bucket_name, Prefix=prefix)``



In [12]:
Bucket=bucket_name
response=s3_client.list_objects_v2(Bucket=Bucket,Prefix='images/') # we uploaded the file here

In [15]:
for content in response['Contents']:
    print(content['Key'])

images/cars.png
images/emotion_faces.jpg


In [19]:
response['Contents'] ## we cam see the object is already there 

[{'Key': 'images/cars.png',
  'LastModified': datetime.datetime(2024, 7, 13, 5, 8, 7, tzinfo=tzutc()),
  'ETag': '"f690f4c51ecb0719848563f508c1408a"',
  'Size': 675682,
  'StorageClass': 'STANDARD'}]

## Downloading or Retriving the uploaded file 

For this we will use the method `s3_client.download_file()`

**Parameters**

- ``Bucket``(**string**): Name of the Bucket where you want to upload the file. In above example we created a unique-bucket

- ``Key``(**string**): key file name you want to download. It is the `Key that you assign while uploading file` 

- ``Filename``(**string**): Name of the downloaded file. Can be a basename or full path. 

In [24]:
##downlaod the file now 

Key=img_key
Bucket=bucket_name
downloaded_folder=f'data/downloaded/{bucket_name}'

if not os.path.exists(downloaded_folder):
    os.makedirs(downloaded_folder)

downloaded_filename=os.path.join(downloaded_folder,Key)

s3_client.download_file(Bucket=Bucket,
                        Key=Key,
                        Filename='cars.png')

In [21]:
downloaded_filename

'data/downloaded/akashghimireofficial/images/cars.png'